In [1]:
%pwd
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook
import warnings ; warnings.filterwarnings(action='ignore')

In [4]:
URL = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008'
driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)

In [3]:
import requests

headers = {'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}

In [ ]:
url_list=[]

In [15]:
# 첫 화면 url 크롤링
urls=driver.find_elements_by_css_selector(".prd_info>a")

for url in urls:
    url=url.get_attribute('href')
    url_list.append(url)
    
print(len(url_list))
url_list

24


['https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000154787&dispCatNo=1000001000100010004&trackingCd=Cat100000100010008_Small&curation&egcode&rccode&egrankcode',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000155065&dispCatNo=1000001000100080001&trackingCd=Cat100000100010008_Small&curation&egcode&rccode&egrankcode',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000155253&dispCatNo=1000001000100080001&trackingCd=Cat100000100010008_Small&curation&egcode&rccode&egrankcode',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000137180&dispCatNo=1000001000100080001&trackingCd=Cat100000100010008_Small&curation&egcode&rccode&egrankcode',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000141704&dispCatNo=1000001000100080001&trackingCd=Cat100000100010008_Small&curation&egcode&rccode&egrankcode',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A00000

In [16]:
# 2~n번째 화면 url 크롤링
for i in range(1, 10):
    driver.find_element_by_xpath('//*[@id="Container"]/div[2]/a[' + str(i) + ']').click()
    time.sleep(3)
    
    urls=driver.find_elements_by_css_selector(".prd_info>a")
    
    for url in urls:
        url=url.get_attribute('href')
        url_list.append(url)
    

In [52]:
# url 저장
olive_young_skin_url = pd.DataFrame(url_list)
olive_young_skin_url.to_csv("url_list.csv", index=False)

In [53]:
url_list.columns= ["순번", "링크"]

In [7]:
url_list = pd.read_csv("url_list_0823.csv")

In [191]:
# 정보 담을 리스트
name = []
evaluation = []
count_add_score= []

ratio_score = []

ratio_category = []

tag = []

In [192]:
# 상품 리뷰정보 크롤링

# url_list 불러오기
for i in range(100):
    URL = url_list['링크'][i]
    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.get(url=URL)
    
    time.sleep(2)
    
    # 상품명
    get_name = driver.find_element_by_xpath('//*[@id="Contents"]/div[2]/div[2]/div/p[2]')
    name.append(get_name.text)

    # 리뷰 버튼
    review_button = driver.find_element_by_xpath('//*[@id="reviewInfo"]/a')
    review_button.click()
    
    time.sleep(3)

    # 평가
    get_eval = driver.find_elements_by_css_selector(".grade_img")
    for j in get_eval:
        evaluation.append(j.text)

    # 건수, 점수
    get_count = driver.find_elements_by_css_selector(".star_area")
    for j in get_count:
        count_add_score.append(j.text)

    # 5~1점 비율
    get_ratio = driver.find_elements_by_css_selector(".graph_list")
    for j in get_ratio:
        ratio_score.append(j.text)
    
    # 피부타입, 피부고민, 자극도 비율
    get_ratio_cate = driver.find_elements_by_css_selector(".poll_all.clrfix")
    for j in get_ratio_cate:
        ratio_category.append(j.text)
    

    # 해시태그
    keywords = driver.find_elements_by_css_selector(".reviewCate>ul>li")
    keywords_list = []
    for keyword in keywords:
        keyword = keyword.get_attribute('data-keyword')
        keywords_list.append(keyword)
    tag.append(keywords_list)

    driver.close()

In [193]:
import pandas as pd
product_review_info = pd.DataFrame(list(zip(name, evaluation, count_add_score, ratio_score, ratio_category, tag)), 
             columns = ['상품명', '평가', '리뷰수와점수', '점수비율', '카테고리비율', '태그'])

In [195]:
product_review_info.to_csv("product_review_info_100.csv", index=False)

In [196]:
pd.read_csv("product_review_info_100.csv")

,상품명,평가,리뷰수와점수,점수비율,카테고리비율,태그
0,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건\n4.6점",75%\n5점\n18%\n4점\n5%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n20%\n복합성에 좋아요\n64%\n지성에 좋아요\n17...,"['ALL', '8809738592137|여름', '8809738592137|여드름..."
1,아누아 어성초 77 수딩토너 기획세트 (350ml+40ml+화장솜60매),최고,총 354 건\n4.8점,86%\n5점\n11%\n4점\n2%\n3점\n0%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n16%\n복합성에 좋아요\n61%\n지성에 좋아요\n23...,"['ALL', '8809640731525|뾰루지,8809640731525|여드름,8..."
2,아비브 어성초 카밍 토너 스킨부스터 더블 기획,최고,"총 1,172 건\n4.6점",74%\n5점\n19%\n4점\n6%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n18%\n복합성에 좋아요\n75%\n지성에 좋아요\n7%...,"['ALL', '8809738592540|여름철', '8809738592540|기획..."
3,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),최고,"총 4,361 건\n4.8점",83%\n5점\n14%\n4점\n3%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n22%\n복합성에 좋아요\n62%\n지성에 좋아요\n16...,"['ALL', '8809657115431|사계절', '8809657115431|여드..."
4,★세븐틴 승관 토너★브링그린 티트리 시카 수딩 토너 510mL(대용량),최고,"총 8,922 건\n4.7점",79%\n5점\n16%\n4점\n4%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n13%\n복합성에 좋아요\n59%\n지성에 좋아요\n28...,"['ALL', '8809738600436|여름', '8809738600436|드름,..."
...,...,...,...,...,...,...
93,마몽드 로즈워터토너 500ml 화장솜 기획 (NEW),최고,"총 1,221 건\n4.8점",82%\n5점\n15%\n4점\n2%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n14%\n복합성에 좋아요\n55%\n지성에 좋아요\n31...,"['ALL', '8809585106105|사계절', '8809585106105|뾰루..."
94,피지오겔 DMT 로션 200ml,최고,"총 5,950 건\n4.7점",79%\n5점\n17%\n4점\n3%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n47%\n복합성에 좋아요\n53%\n지성에 좋아요\n1%...,"['ALL', '8806105402426|겨울,8806105402426|겨울철', ..."
95,더바디샵 티트리 오일 20ML,최고,총 94 건\n4.8점,83%\n5점\n14%\n4점\n2%\n3점\n1%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n31%\n복합성에 좋아요\n61%\n지성에 좋아요\n8%...,"['ALL', '5028197972530|드름,5028197972530|뾰루지,50..."
96,수려한 비책진생 특별 기획세트 (수액150ml+유액130ml+크림30ml),최고,총 77 건\n4.8점,88%\n5점\n10%\n4점\n0%\n3점\n0%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n15%\n복합성에 좋아요\n68%\n지성에 좋아요\n17...,"['ALL', '8801051865462|구성', '8801051865462|화장품..."


In [74]:
'''
# 리뷰 텍스트

review_txts = driver.find_elements_by_css_selector(".txt_inner")
    
review_txt_list = []
for review_txt in review_txts:
    review_txt = review_txt.text
    review_txt_list.append(review_txt)
'''

In [8]:
url_list

,순번,링크
0,0,https://www.oliveyoung.co.kr/store/goods/getGo...
1,1,https://www.oliveyoung.co.kr/store/goods/getGo...
2,2,https://www.oliveyoung.co.kr/store/goods/getGo...
3,3,https://www.oliveyoung.co.kr/store/goods/getGo...
4,4,https://www.oliveyoung.co.kr/store/goods/getGo...
...,...,...
618,618,https://www.oliveyoung.co.kr/store/goods/getGo...
619,619,https://www.oliveyoung.co.kr/store/goods/getGo...
620,620,https://www.oliveyoung.co.kr/store/goods/getGo...
621,621,https://www.oliveyoung.co.kr/store/goods/getGo...


In [59]:
count = 0


URL = url_list['링크'][24]
driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)
    
time.sleep(2)
    
# 상품명
name = []
get_name = driver.find_element_by_xpath('//*[@id="Contents"]/div[2]/div[2]/div/p[2]')
name.append(get_name.text)

# 리뷰 버튼
review_button = driver.find_element_by_xpath('//*[@id="reviewInfo"]/a')
review_button.click()
    
time.sleep(3)

reviewer_info_list = []
review_txt_list = []
while True:
    
    if (count == 0):
        for i in range(0, 10):
    
            if (i == 0):
                # 리뷰어 정보
                reviewer_info = driver.find_elements_by_css_selector(".user.clrfix")
                for info in reviewer_info:
                    reviewer_info_list.append(info.text)
            
                # 리뷰 정보
                review_txts = driver.find_elements_by_css_selector(".review_cont")
                for review_txt in review_txts:
                    review_txt_list.append(review_txt.text)
        
            else :
                driver.find_element_by_xpath('//*[@id="gdasContentsArea"]/div/div[6]/a[' + str(i) + ']').click()
                time.sleep(3)
                
                reviewer_info = driver.find_elements_by_css_selector(".user.clrfix")
                for info in reviewer_info:
                    reviewer_info_list.append(info.text)
            
                review_txts = driver.find_elements_by_css_selector(".review_cont")
                for review_txt in review_txts:
                    review_txt_list.append(review_txt.text)
            
        driver.find_element_by_xpath('//*[@id="gdasContentsArea"]/div/div[6]/a[10]').click()
        
        count += 1
        
        time.sleep(2)
    
    else :
        print("성공")
        for j in range(1, 11):
            print(j)
    
            if (j == 1):
                print(j)
                reviewer_info = driver.find_elements_by_css_selector(".user.clrfix")
                for info in reviewer_info:
                    reviewer_info_list.append(info.text)
            
                review_txts = driver.find_elements_by_css_selector(".review_cont")
                for review_txt in review_txts:
                    review_txt_list.append(review_txt.text)
        
            else :
                print(j)
                time.sleep(4)
                driver.find_element_by_xpath('//*[@id="gdasContentsArea"]/div/div[6]/a[' + str(j) + ']').click()
                #driver.find_element_by_xpath('//*[@id="gdasContentsArea"]/div/div[7]/a[2]').click()
                time.sleep(3)
    
                reviewer_info = driver.find_elements_by_css_selector(".user.clrfix")
                for info in reviewer_info:
                    reviewer_info_list.append(info.text)
            
                review_txts = driver.find_elements_by_css_selector(".review_cont")
                for review_txt in review_txts:
                    review_txt_list.append(review_txt.text)
            
        driver.find_element_by_xpath('//*[@id="gdasContentsArea"]/div/div[6]/a[11]').click()
        
        time.sleep(2)
        


성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
성공
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="gdasContentsArea"]/div/div[6]/a[11]"}
  (Session info: chrome=102.0.5005.115)


In [60]:
# 데이터프레임 생성
name = name * len(reviewer_info_list)

product_review_text = pd.DataFrame(list(zip(name, reviewer_info_list, review_txt_list)), 
             columns = ['상품명', '리뷰어정보', '리뷰정보'])

product_review_text

,상품명,리뷰어정보,리뷰정보
0,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),뚠땐뚠\n복합성\n쿨톤\n민감성\n트러블\n한달사용리뷰,5점만점에 5점\n2021.07.08\n🌿향에 민감한 반응을 보여 화장품을 고를때 ...
1,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),tnaakrgl****\n복합성\n웜톤\n각질\n잡티,5점만점에 5점\n2020.12.30\n처음에는 로션을 사기위해서 검색창에 로션을 ...
2,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),갈웜딥극건성입니당\nTOP 100\n네일 분야 탑리뷰어\n약건성\n가을웜톤\n각질\...,5점만점에 5점\n2022.05.26\n성분이 정말 정말 좋은가봐요!! \n이거 쓰...
3,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),바나나챠챠\nTOP 900\n복합성\n쿨톤\n각질\n모공\n재구매,5점만점에 5점\n2022.05.23\n피부가 예민한 타입인데도 이 제품을 사용하면...
4,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),hymn\nTOP 200\n클렌징 분야 탑리뷰어\n건성\n웜톤\n각질\n미백,5점만점에 5점\n2022.03.16 선물\n민감성+지성피부인 동생에게 선물로 사주...
...,...,...,...
995,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),cntns****,5점만점에 4점\n2020.05.08
996,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),프링\n건성\n쿨톤\n모공\n미백,5점만점에 5점\n2020.05.08 매장
997,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),dlgksruf****,5점만점에 5점\n2020.05.07 매장\n만오천원에 스킨로션 마스크팩 3장까지 ...
998,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),kjy02****,5점만점에 4점\n2020.05.05 매장


In [61]:
product_review_text.to_csv("product_review_text_24.csv", index=False)

In [62]:
pd.read_csv("product_review_text_24.csv")

,상품명,리뷰어정보,리뷰정보
0,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),뚠땐뚠\n복합성\n쿨톤\n민감성\n트러블\n한달사용리뷰,5점만점에 5점\n2021.07.08\n🌿향에 민감한 반응을 보여 화장품을 고를때 ...
1,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),tnaakrgl****\n복합성\n웜톤\n각질\n잡티,5점만점에 5점\n2020.12.30\n처음에는 로션을 사기위해서 검색창에 로션을 ...
2,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),갈웜딥극건성입니당\nTOP 100\n네일 분야 탑리뷰어\n약건성\n가을웜톤\n각질\...,5점만점에 5점\n2022.05.26\n성분이 정말 정말 좋은가봐요!! \n이거 쓰...
3,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),바나나챠챠\nTOP 900\n복합성\n쿨톤\n각질\n모공\n재구매,5점만점에 5점\n2022.05.23\n피부가 예민한 타입인데도 이 제품을 사용하면...
4,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),hymn\nTOP 200\n클렌징 분야 탑리뷰어\n건성\n웜톤\n각질\n미백,5점만점에 5점\n2022.03.16 선물\n민감성+지성피부인 동생에게 선물로 사주...
...,...,...,...
995,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),cntns****,5점만점에 4점\n2020.05.08
996,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),프링\n건성\n쿨톤\n모공\n미백,5점만점에 5점\n2020.05.08 매장
997,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),dlgksruf****,5점만점에 5점\n2020.05.07 매장\n만오천원에 스킨로션 마스크팩 3장까지 ...
998,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),kjy02****,5점만점에 4점\n2020.05.05 매장


In [63]:
df0 = pd.read_csv("product_review_text_10.csv")
df1 = pd.read_csv("product_review_text_11.csv")
df2 = pd.read_csv("product_review_text_13.csv")
df3 = pd.read_csv("product_review_text_15.csv")
df4 = pd.read_csv("product_review_text_16.csv")
df5 = pd.read_csv("product_review_text_17.csv")
df6 = pd.read_csv("product_review_text_18.csv")
df7 = pd.read_csv("product_review_text_20.csv")
df8 = pd.read_csv("product_review_text_24.csv")

In [64]:
temp = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8], axis=0)

In [65]:
temp

,상품명,리뷰어정보,리뷰정보
0,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),수호부인\n건성\n각질\n미백,5점만점에 5점\n2019.12.16 올영체험단\n올인원 잘 만들어진 제품!!\n피...
1,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),올그린\nTOP 500\n건성\n웜톤\n모공\n미백\n한달사용리뷰,5점만점에 4점\n2021.10.14\n[옵션] 기획\n피부타입\n복합성에 좋아요\...
2,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),몽실퐁퐁S2\nTOP 900\n복합성\n웜톤\n모공\n블랙헤드,5점만점에 5점\n2021.06.18\n[옵션] 기획\n피부타입\n복합성에 좋아요\...
3,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),호빵년98\nTOP 400\n지성\n웜톤\n각질\n모공,5점만점에 5점\n2020.09.18 매장\n피부타입\n복합성에 좋아요\n피부고민\...
4,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),뚜딘\n복합성\n웜톤\n모공\n민감성,5점만점에 5점\n2021.02.18\n[옵션] 기획\n피부타입\n복합성에 좋아요\...
...,...,...,...
995,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),cntns****,5점만점에 4점\n2020.05.08
996,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),프링\n건성\n쿨톤\n모공\n미백,5점만점에 5점\n2020.05.08 매장
997,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),dlgksruf****,5점만점에 5점\n2020.05.07 매장\n만오천원에 스킨로션 마스크팩 3장까지 ...
998,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),kjy02****,5점만점에 4점\n2020.05.05 매장


In [76]:
# 리뷰 정보 스플릿
def text_split(df):
    try:
    
        if (len(df['리뷰정보'].split('\n')) == 2):
            df['평점'] = df['리뷰정보'].split('\n')[0]
            df['작성날짜'] = df['리뷰정보'].split('\n')[1]
    
        else:
            if (df['리뷰정보'].split('\n')[2] == ''):
                df['평점'] = df['리뷰정보'].split('\n')[0]
                df['작성날짜'] = df['리뷰정보'].split('\n')[1]
                sample = df['리뷰정보'].split('\n')[2:-2]
                df['리뷰'] = '\n'.join(sample)
                df['도움수'] = df['리뷰정보'].split('\n')[-2]
        
            elif (df['리뷰정보'].split('\n')[2][0] == '피'):
                df['평점'] = df['리뷰정보'].split('\n')[0]
                df['작성날짜'] = df['리뷰정보'].split('\n')[1]
                df['피부타입'] = df['리뷰정보'].split('\n')[3]
                df['피부고민'] = df['리뷰정보'].split('\n')[5]
                df['자극도'] = df['리뷰정보'].split('\n')[7]
    
                sample = df['리뷰정보'].split('\n')[8:-2]
                df['리뷰'] = '\n'.join(sample)
                df['도움수'] = df['리뷰정보'].split('\n')[-2]
        
            else:
                df['평점'] = df['리뷰정보'].split('\n')[0]
                df['작성날짜'] = df['리뷰정보'].split('\n')[1]
                sample = df['리뷰정보'].split('\n')[2:-2]
                df['리뷰'] = '\n'.join(sample)
                df['도움수'] = df['리뷰정보'].split('\n')[-2]
    except:
        pass
    return df

In [74]:
temp['리뷰정보'].iloc[64]

'5점만점에 5점\n2021.07.12 매장\n[옵션] 기획\n피부타입\n복합성에 좋아요\n피부고민\n보습에 좋아요\n자극도\n자극없이 순해요\n○ 남자분들은 올인원 많이 쓰시더라구요! 남자친구도 새로 사야한다는데 뭐가 좋은지는 잘 모르더라구요..저도 올인원 제품은 안써봐서 뭐가 좋은지 몰라서 추천을 못해주겠더라구요ㅠ\n그때 닥터지에도 올인원 제품이 있길래 무슨 자신감인지는 모르겠지만 추천해줬습니다ㅋㅋㅋ\n저도 전에 닥터지 크림 썼었고 클렌징젤도 잘 쓰고있어서 이것도 자신있게 써보라고 추천해줬던 것 같습니다ㅎㅎ\n\n○ 남자친구 피부는 지성인데 파워 지성까지는 아닌?타입입니다.지성인 피부도 무난하게 쓰기좋다고 하는것 보면 닥터지 제품 특성상 복합성이신분들에게도 잘 맞을 것 같아요!아!보습면에서요!\n건성이신분들은..여름에는 쓸만할 것 같다고는 한데 겨울에는 잘 모르겠다고 하더라구요..\n\n○ 저도 들은말을 그대로 적는거라 디테일이 부족하네요ㅠㅠ죄송합니다ㅠㅠ\n이 리뷰가 도움이 돼요! 0\n신고하기'

In [77]:
df_temp = temp.apply(text_split, axis=1)

In [80]:
df_temp

,도움수,리뷰,리뷰어정보,리뷰정보,상품명,자극도,작성날짜,평점,피부고민,피부타입
0,이 리뷰가 도움이 돼요! 18,올인원 잘 만들어진 제품!!\n피부타입\n건성에 좋아요\n피부고민\n보습에 좋아요\...,수호부인\n건성\n각질\n미백,5점만점에 5점\n2019.12.16 올영체험단\n올인원 잘 만들어진 제품!!\n피...,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),NaN,2019.12.16 올영체험단,5점만점에 5점,NaN,NaN
1,이 리뷰가 도움이 돼요! 7,[옵션] 기획\n피부타입\n복합성에 좋아요\n피부고민\n진정에 좋아요\n자극도\n자...,올그린\nTOP 500\n건성\n웜톤\n모공\n미백\n한달사용리뷰,5점만점에 4점\n2021.10.14\n[옵션] 기획\n피부타입\n복합성에 좋아요\...,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),NaN,2021.10.14,5점만점에 4점,NaN,NaN
2,이 리뷰가 도움이 돼요! 10,[옵션] 기획\n피부타입\n복합성에 좋아요\n피부고민\n보습에 좋아요\n자극도\n자...,몽실퐁퐁S2\nTOP 900\n복합성\n웜톤\n모공\n블랙헤드,5점만점에 5점\n2021.06.18\n[옵션] 기획\n피부타입\n복합성에 좋아요\...,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),NaN,2021.06.18,5점만점에 5점,NaN,NaN
3,이 리뷰가 도움이 돼요! 24,‼️남자친구 주려고 매장에서 이것저것 다테스트 해보고 골랐어요! 남자친구는➡️ 건성...,호빵년98\nTOP 400\n지성\n웜톤\n각질\n모공,5점만점에 5점\n2020.09.18 매장\n피부타입\n복합성에 좋아요\n피부고민\...,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),자극없이 순해요,2020.09.18 매장,5점만점에 5점,보습에 좋아요,복합성에 좋아요
4,이 리뷰가 도움이 돼요! 12,[옵션] 기획\n피부타입\n복합성에 좋아요\n피부고민\n진정에 좋아요\n자극도\n자...,뚜딘\n복합성\n웜톤\n모공\n민감성,5점만점에 5점\n2021.02.18\n[옵션] 기획\n피부타입\n복합성에 좋아요\...,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),NaN,2021.02.18,5점만점에 5점,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,cntns****,5점만점에 4점\n2020.05.08,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),NaN,2020.05.08,5점만점에 4점,NaN,NaN
996,NaN,NaN,프링\n건성\n쿨톤\n모공\n미백,5점만점에 5점\n2020.05.08 매장,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),NaN,2020.05.08 매장,5점만점에 5점,NaN,NaN
997,이 리뷰가 도움이 돼요! 0,만오천원에 스킨로션 마스크팩 3장까지 구성이 넘조아유\n만오천원에 스킨로션 마스크팩...,dlgksruf****,5점만점에 5점\n2020.05.07 매장\n만오천원에 스킨로션 마스크팩 3장까지 ...,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),NaN,2020.05.07 매장,5점만점에 5점,NaN,NaN
998,NaN,NaN,kjy02****,5점만점에 4점\n2020.05.05 매장,싸이닉 더 심플 스킨케어 2종 기획 (토너145mlx2+로션145ml),NaN,2020.05.05 매장,5점만점에 4점,NaN,NaN


In [81]:
df_temp.to_csv("olive_review_11to20_0616.csv", index=False, encoding='utf-8-sig')

In [86]:
df_temp.iloc[24]['리뷰']

'[옵션] 기획\n피부타입\n건성에 좋아요\n피부고민\n보습에 좋아요\n자극도\n자극없이 순해요\n신랑 기존에 사용하던 올인원 로션이 다 떨어졌는데\n유튜브 볼때마다 광고하는 노래가 자꾸 떠올라서 이번에\n새로운 시도로 구매해봤어요!\n\n일단 케이스 자체는 사용하기 편리하고 펌핑형식으로 되어있다보니 위생적으로도 먼지나 이물질이 들어가지 않을것 같아서 좋았어요 :) 함께 들어있는 작은 사이즈는 여행다니거나\n휴대하기 딱 적당한 크기라서 요긴하게 쓸것같구요~\n\n일단 향은 여성제품들과는 다르게 남성의 향이 살짝 나지만\n그렇게 강한편은 아니였고 로션과 토너의 중간정도 되는 점도로 적은 양이지만 쉽게 얼굴에 넓게 펴발리는 편이에요!\n그렇다보니 춥고 엄청 건조한 계절에는 단독사용 했을때 조금 당김이 있을것 같아서 봄부터 가을까지 추천드릴게요~\n원래 신랑피부는 좋은편이라 광고처럼 진정효과가 있다고 쓰기는 어렵지만 여전히 피부가 좋으니 잘 맞는거같아요!'

In [417]:
info = pd.read_csv("product_review_info_100.csv")

In [418]:
info

,상품명,평가,리뷰수와점수,점수비율,카테고리비율,태그
0,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건\n4.6점",75%\n5점\n18%\n4점\n5%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n20%\n복합성에 좋아요\n64%\n지성에 좋아요\n17...,"['ALL', '8809738592137|여름', '8809738592137|여드름..."
1,아누아 어성초 77 수딩토너 기획세트 (350ml+40ml+화장솜60매),최고,총 354 건\n4.8점,86%\n5점\n11%\n4점\n2%\n3점\n0%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n16%\n복합성에 좋아요\n61%\n지성에 좋아요\n23...,"['ALL', '8809640731525|뾰루지,8809640731525|여드름,8..."
2,아비브 어성초 카밍 토너 스킨부스터 더블 기획,최고,"총 1,172 건\n4.6점",74%\n5점\n19%\n4점\n6%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n18%\n복합성에 좋아요\n75%\n지성에 좋아요\n7%...,"['ALL', '8809738592540|여름철', '8809738592540|기획..."
3,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),최고,"총 4,361 건\n4.8점",83%\n5점\n14%\n4점\n3%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n22%\n복합성에 좋아요\n62%\n지성에 좋아요\n16...,"['ALL', '8809657115431|사계절', '8809657115431|여드..."
4,★세븐틴 승관 토너★브링그린 티트리 시카 수딩 토너 510mL(대용량),최고,"총 8,922 건\n4.7점",79%\n5점\n16%\n4점\n4%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n13%\n복합성에 좋아요\n59%\n지성에 좋아요\n28...,"['ALL', '8809738600436|여름', '8809738600436|드름,..."
...,...,...,...,...,...,...
93,마몽드 로즈워터토너 500ml 화장솜 기획 (NEW),최고,"총 1,221 건\n4.8점",82%\n5점\n15%\n4점\n2%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n14%\n복합성에 좋아요\n55%\n지성에 좋아요\n31...,"['ALL', '8809585106105|사계절', '8809585106105|뾰루..."
94,피지오겔 DMT 로션 200ml,최고,"총 5,950 건\n4.7점",79%\n5점\n17%\n4점\n3%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n47%\n복합성에 좋아요\n53%\n지성에 좋아요\n1%...,"['ALL', '8806105402426|겨울,8806105402426|겨울철', ..."
95,더바디샵 티트리 오일 20ML,최고,총 94 건\n4.8점,83%\n5점\n14%\n4점\n2%\n3점\n1%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n31%\n복합성에 좋아요\n61%\n지성에 좋아요\n8%...,"['ALL', '5028197972530|드름,5028197972530|뾰루지,50..."
96,수려한 비책진생 특별 기획세트 (수액150ml+유액130ml+크림30ml),최고,총 77 건\n4.8점,88%\n5점\n10%\n4점\n0%\n3점\n0%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n15%\n복합성에 좋아요\n68%\n지성에 좋아요\n17...,"['ALL', '8801051865462|구성', '8801051865462|화장품..."


In [421]:
# 상품 리뷰 요약정보 스플릿
def info_split(df):
    
    df['리뷰수'] = df['리뷰수와점수'].split('\n')[0]
    df['평균평점'] = df['리뷰수와점수'].split('\n')[1]
    
    df['5점비율'] = df['점수비율'].split('\n')[0]
    df['4점비율'] = df['점수비율'].split('\n')[2]
    df['3점비율'] = df['점수비율'].split('\n')[4]
    df['2점비율'] = df['점수비율'].split('\n')[6]
    df['1점비율'] = df['점수비율'].split('\n')[8]
    
    df['피부타입_건성_좋은비율'] = df['카테고리비율'].split('\n')[2]
    df['피부타입_복합성_좋은비율'] = df['카테고리비율'].split('\n')[4]
    df['피부타입_지성_좋은비율'] = df['카테고리비율'].split('\n')[6]
    
    df['피부고민_보습_좋은비율'] = df['카테고리비율'].split('\n')[9]
    df['피부고민_진정_좋은비율'] = df['카테고리비율'].split('\n')[11]
    df['피부고민_주름/미백_좋은비율'] = df['카테고리비율'].split('\n')[13]
    
    df['자극도_순함'] = df['카테고리비율'].split('\n')[16]
    df['자극도_보통'] = df['카테고리비율'].split('\n')[18]
    df['자극도_느껴짐'] = df['카테고리비율'].split('\n')[20]
    
    return df

In [425]:
df_info = info.apply(info_split, axis=1)

In [428]:
df_info

,상품명,평가,리뷰수와점수,점수비율,카테고리비율,태그,리뷰수,평균평점,5점비율,4점비율,...,1점비율,피부타입_건성_좋은비율,피부타입_복합성_좋은비율,피부타입_지성_좋은비율,피부고민_보습_좋은비율,피부고민_진정_좋은비율,피부고민_주름/미백_좋은비율,자극도_순함,자극도_보통,자극도_느껴짐
0,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건\n4.6점",75%\n5점\n18%\n4점\n5%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n20%\n복합성에 좋아요\n64%\n지성에 좋아요\n17...,"['ALL', '8809738592137|여름', '8809738592137|여드름...","총 1,632 건",4.6점,75%,18%,...,1%,20%,64%,17%,55%,45%,1%,79%,21%,2%
1,아누아 어성초 77 수딩토너 기획세트 (350ml+40ml+화장솜60매),최고,총 354 건\n4.8점,86%\n5점\n11%\n4점\n2%\n3점\n0%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n16%\n복합성에 좋아요\n61%\n지성에 좋아요\n23...,"['ALL', '8809640731525|뾰루지,8809640731525|여드름,8...",총 354 건,4.8점,86%,11%,...,1%,16%,61%,23%,15%,85%,0%,82%,18%,1%
2,아비브 어성초 카밍 토너 스킨부스터 더블 기획,최고,"총 1,172 건\n4.6점",74%\n5점\n19%\n4점\n6%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n18%\n복합성에 좋아요\n75%\n지성에 좋아요\n7%...,"['ALL', '8809738592540|여름철', '8809738592540|기획...","총 1,172 건",4.6점,74%,19%,...,1%,18%,75%,7%,11%,89%,0%,70%,30%,0%
3,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),최고,"총 4,361 건\n4.8점",83%\n5점\n14%\n4점\n3%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n22%\n복합성에 좋아요\n62%\n지성에 좋아요\n16...,"['ALL', '8809657115431|사계절', '8809657115431|여드...","총 4,361 건",4.8점,83%,14%,...,0%,22%,62%,16%,39%,61%,1%,83%,17%,1%
4,★세븐틴 승관 토너★브링그린 티트리 시카 수딩 토너 510mL(대용량),최고,"총 8,922 건\n4.7점",79%\n5점\n16%\n4점\n4%\n3점\n1%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n13%\n복합성에 좋아요\n59%\n지성에 좋아요\n28...,"['ALL', '8809738600436|여름', '8809738600436|드름,...","총 8,922 건",4.7점,79%,16%,...,1%,13%,59%,28%,13%,87%,1%,61%,36%,3%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,마몽드 로즈워터토너 500ml 화장솜 기획 (NEW),최고,"총 1,221 건\n4.8점",82%\n5점\n15%\n4점\n2%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n14%\n복합성에 좋아요\n55%\n지성에 좋아요\n31...,"['ALL', '8809585106105|사계절', '8809585106105|뾰루...","총 1,221 건",4.8점,82%,15%,...,0%,14%,55%,31%,6%,94%,0%,28%,56%,16%
94,피지오겔 DMT 로션 200ml,최고,"총 5,950 건\n4.7점",79%\n5점\n17%\n4점\n3%\n3점\n0%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n47%\n복합성에 좋아요\n53%\n지성에 좋아요\n1%...,"['ALL', '8806105402426|겨울,8806105402426|겨울철', ...","총 5,950 건",4.7점,79%,17%,...,0%,47%,53%,1%,62%,14%,24%,51%,49%,0%
95,더바디샵 티트리 오일 20ML,최고,총 94 건\n4.8점,83%\n5점\n14%\n4점\n2%\n3점\n1%\n2점\n0%\n1점,피부타입\n건성에 좋아요\n31%\n복합성에 좋아요\n61%\n지성에 좋아요\n8%...,"['ALL', '5028197972530|드름,5028197972530|뾰루지,50...",총 94 건,4.8점,83%,14%,...,0%,31%,61%,8%,52%,48%,1%,74%,26%,1%
96,수려한 비책진생 특별 기획세트 (수액150ml+유액130ml+크림30ml),최고,총 77 건\n4.8점,88%\n5점\n10%\n4점\n0%\n3점\n0%\n2점\n1%\n1점,피부타입\n건성에 좋아요\n15%\n복합성에 좋아요\n68%\n지성에 좋아요\n17...,"['ALL', '8801051865462|구성', '8801051865462|화장품...",총 77 건,4.8점,88%,10%,...,1%,15%,68%,17%,14%,84%,2%,21%,46%,33%


In [429]:
df_info.drop(['리뷰수와점수','점수비율','카테고리비율'], axis=1, inplace=True)

In [431]:
df_info.to_csv("product_review_info_preprocessing_0824.csv", index=False)

In [435]:
review_info_merge = pd.read_csv("product_review_info_preprocessing_0824.csv")

In [436]:
review_info_merge

,상품명,평가,태그,리뷰수,평균평점,5점비율,4점비율,3점비율,2점비율,1점비율,피부타입_건성_좋은비율,피부타입_복합성_좋은비율,피부타입_지성_좋은비율,피부고민_보습_좋은비율,피부고민_진정_좋은비율,피부고민_주름/미백_좋은비율,자극도_순함,자극도_보통,자극도_느껴짐
0,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"['ALL', '8809738592137|여름', '8809738592137|여드름...","총 1,632 건",4.6점,75%,18%,5%,1%,1%,20%,64%,17%,55%,45%,1%,79%,21%,2%
1,아누아 어성초 77 수딩토너 기획세트 (350ml+40ml+화장솜60매),최고,"['ALL', '8809640731525|뾰루지,8809640731525|여드름,8...",총 354 건,4.8점,86%,11%,2%,0%,1%,16%,61%,23%,15%,85%,0%,82%,18%,1%
2,아비브 어성초 카밍 토너 스킨부스터 더블 기획,최고,"['ALL', '8809738592540|여름철', '8809738592540|기획...","총 1,172 건",4.6점,74%,19%,6%,1%,1%,18%,75%,7%,11%,89%,0%,70%,30%,0%
3,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),최고,"['ALL', '8809657115431|사계절', '8809657115431|여드...","총 4,361 건",4.8점,83%,14%,3%,0%,0%,22%,62%,16%,39%,61%,1%,83%,17%,1%
4,★세븐틴 승관 토너★브링그린 티트리 시카 수딩 토너 510mL(대용량),최고,"['ALL', '8809738600436|여름', '8809738600436|드름,...","총 8,922 건",4.7점,79%,16%,4%,1%,1%,13%,59%,28%,13%,87%,1%,61%,36%,3%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,마몽드 로즈워터토너 500ml 화장솜 기획 (NEW),최고,"['ALL', '8809585106105|사계절', '8809585106105|뾰루...","총 1,221 건",4.8점,82%,15%,2%,0%,0%,14%,55%,31%,6%,94%,0%,28%,56%,16%
94,피지오겔 DMT 로션 200ml,최고,"['ALL', '8806105402426|겨울,8806105402426|겨울철', ...","총 5,950 건",4.7점,79%,17%,3%,0%,0%,47%,53%,1%,62%,14%,24%,51%,49%,0%
95,더바디샵 티트리 오일 20ML,최고,"['ALL', '5028197972530|드름,5028197972530|뾰루지,50...",총 94 건,4.8점,83%,14%,2%,1%,0%,31%,61%,8%,52%,48%,1%,74%,26%,1%
96,수려한 비책진생 특별 기획세트 (수액150ml+유액130ml+크림30ml),최고,"['ALL', '8801051865462|구성', '8801051865462|화장품...",총 77 건,4.8점,88%,10%,0%,0%,1%,15%,68%,17%,14%,84%,2%,21%,46%,33%


In [434]:
df_temp

,도움수,리뷰,리뷰어정보,리뷰정보,상품명,자극도,작성날짜,평점,피부고민,피부타입
0,이 리뷰가 도움이 돼요! 15,이브랜드는 올리브영 입점전부터 써본브랜드입니다\n수분크림구매시에 성분이 중요한데요\...,화장품분석녀\nTOP 400\n메이크업 분야 탑리뷰어\n복합성\n웜톤\n각질\n모공,5점만점에 5점\n2021.08.10\n피부타입\n복합성에 좋아요\n피부고민\n보습...,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.08.10,5점만점에 5점,보습에 좋아요,복합성에 좋아요
1,이 리뷰가 도움이 돼요! 8,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n1+1행사한다...,앙꼬빵\n지성\n모공\n블랙헤드,5점만점에 5점\n2021.08.09\n피부타입\n지성에 좋아요\n피부고민\n보습에...,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.08.09,5점만점에 5점,보습에 좋아요,지성에 좋아요
2,이 리뷰가 도움이 돼요! 9,올리브영 대표 클린뷰티! 라운드랩!\n\n# 라운드랩 자작나무 수분 크림 \n\n✔...,다블리\nTOP 50\n건강/위생용품 분야 탑리뷰어\n복합성\n웜톤\n모공\n블랙헤드,5점만점에 5점\n2021.07.03\n피부타입\n복합성에 좋아요\n피부고민\n진정...,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.07.03,5점만점에 5점,진정에 좋아요,복합성에 좋아요
3,이 리뷰가 도움이 돼요! 11,💚수분감\n진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 1...,라당\nTOP 1000\n지성\n웜톤\n각질\n모공,5점만점에 5점\n2021.02.18\n피부타입\n지성에 좋아요\n피부고민\n보습에...,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.02.18,5점만점에 5점,보습에 좋아요,지성에 좋아요
4,이 리뷰가 도움이 돼요! 1,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,탄산수맛있당\nTOP 900\n건성\n쿨톤\n각질\n미백,5점만점에 5점\n2021.08.13 매장\n피부타입\n복합성에 좋아요\n피부고민\...,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.08.13 매장,5점만점에 5점,보습에 좋아요,복합성에 좋아요
...,...,...,...,...,...,...,...,...,...,...
824,이 리뷰가 도움이 돼요! 0,이 스킨으로 스킨팩하면 확실히 피부가 촉촉해지고 맑아져요!\n※ 해당 리뷰는 원칙적...,ddoc****,5점만점에 5점\n2021.06.08\n피부타입\n복합성에 좋아요\n피부고민\n보습...,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.08,5점만점에 5점,보습에 좋아요,복합성에 좋아요
825,이 리뷰가 도움이 돼요! 0,솔직히 받고 아직 안썼지만 맘에는 듭니다. 잘쓰겄소.\n※ 해당 리뷰는 원칙적으로 ...,ch****,5점만점에 5점\n2021.06.08\n피부타입\n복합성에 좋아요\n피부고민\n진정...,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.08,5점만점에 5점,진정에 좋아요,복합성에 좋아요
826,이 리뷰가 도움이 돼요! 0,제가 써본 토너 중에는 가장 순하고 흡수가 잘되는것 같아요^^\n※ 해당 리뷰는 원...,floray****,5점만점에 4점\n2021.06.08 매장\n피부타입\n건성에 좋아요\n피부고민\n...,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.08 매장,5점만점에 4점,진정에 좋아요,건성에 좋아요
827,이 리뷰가 도움이 돼요! 0,트러블 피부인데 이거는 듬뿍 써도 아무것도 안나요!! 보습효과도 좋아서 3번째 재구...,dndwl****,5점만점에 5점\n2021.06.07\n피부타입\n건성에 좋아요\n피부고민\n보습에...,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.07,5점만점에 5점,보습에 좋아요,건성에 좋아요


In [497]:
df_0824_10 = pd.merge(df_temp, review_info_merge, how='left')

In [498]:
df_0824_10.drop(['리뷰정보'], axis=1, inplace=True)

In [499]:
df_0824_10

,도움수,리뷰,리뷰어정보,상품명,자극도,작성날짜,평점,피부고민,피부타입,평가,...,1점비율,피부타입_건성_좋은비율,피부타입_복합성_좋은비율,피부타입_지성_좋은비율,피부고민_보습_좋은비율,피부고민_진정_좋은비율,피부고민_주름/미백_좋은비율,자극도_순함,자극도_보통,자극도_느껴짐
0,이 리뷰가 도움이 돼요! 15,이브랜드는 올리브영 입점전부터 써본브랜드입니다\n수분크림구매시에 성분이 중요한데요\...,화장품분석녀\nTOP 400\n메이크업 분야 탑리뷰어\n복합성\n웜톤\n각질\n모공,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.08.10,5점만점에 5점,보습에 좋아요,복합성에 좋아요,최고,...,1%,20%,64%,17%,55%,45%,1%,79%,21%,2%
1,이 리뷰가 도움이 돼요! 8,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n1+1행사한다...,앙꼬빵\n지성\n모공\n블랙헤드,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.08.09,5점만점에 5점,보습에 좋아요,지성에 좋아요,최고,...,1%,20%,64%,17%,55%,45%,1%,79%,21%,2%
2,이 리뷰가 도움이 돼요! 9,올리브영 대표 클린뷰티! 라운드랩!\n\n# 라운드랩 자작나무 수분 크림 \n\n✔...,다블리\nTOP 50\n건강/위생용품 분야 탑리뷰어\n복합성\n웜톤\n모공\n블랙헤드,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.07.03,5점만점에 5점,진정에 좋아요,복합성에 좋아요,최고,...,1%,20%,64%,17%,55%,45%,1%,79%,21%,2%
3,이 리뷰가 도움이 돼요! 11,💚수분감\n진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 1...,라당\nTOP 1000\n지성\n웜톤\n각질\n모공,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.02.18,5점만점에 5점,보습에 좋아요,지성에 좋아요,최고,...,1%,20%,64%,17%,55%,45%,1%,79%,21%,2%
4,이 리뷰가 도움이 돼요! 1,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,탄산수맛있당\nTOP 900\n건성\n쿨톤\n각질\n미백,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),자극없이 순해요,2021.08.13 매장,5점만점에 5점,보습에 좋아요,복합성에 좋아요,최고,...,1%,20%,64%,17%,55%,45%,1%,79%,21%,2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14273,이 리뷰가 도움이 돼요! 0,이 스킨으로 스킨팩하면 확실히 피부가 촉촉해지고 맑아져요!\n※ 해당 리뷰는 원칙적...,ddoc****,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.08,5점만점에 5점,보습에 좋아요,복합성에 좋아요,최고,...,0%,32%,54%,14%,60%,40%,0%,83%,17%,0%
14274,이 리뷰가 도움이 돼요! 0,솔직히 받고 아직 안썼지만 맘에는 듭니다. 잘쓰겄소.\n※ 해당 리뷰는 원칙적으로 ...,ch****,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.08,5점만점에 5점,진정에 좋아요,복합성에 좋아요,최고,...,0%,32%,54%,14%,60%,40%,0%,83%,17%,0%
14275,이 리뷰가 도움이 돼요! 0,제가 써본 토너 중에는 가장 순하고 흡수가 잘되는것 같아요^^\n※ 해당 리뷰는 원...,floray****,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.08 매장,5점만점에 4점,진정에 좋아요,건성에 좋아요,최고,...,0%,32%,54%,14%,60%,40%,0%,83%,17%,0%
14276,이 리뷰가 도움이 돼요! 0,트러블 피부인데 이거는 듬뿍 써도 아무것도 안나요!! 보습효과도 좋아서 3번째 재구...,dndwl****,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),자극없이 순해요,2021.06.07,5점만점에 5점,보습에 좋아요,건성에 좋아요,최고,...,0%,32%,54%,14%,60%,40%,0%,83%,17%,0%


In [500]:
df_0824_10.columns

Index(['도움수', '리뷰', '리뷰어정보', '상품명', '자극도', '작성날짜', '평점', '피부고민', '피부타입', '평가',
       '태그', '리뷰수', '평균평점', '5점비율', '4점비율', '3점비율', '2점비율', '1점비율',
       '피부타입_건성_좋은비율', '피부타입_복합성_좋은비율', '피부타입_지성_좋은비율', '피부고민_보습_좋은비율',
       '피부고민_진정_좋은비율', '피부고민_주름/미백_좋은비율', '자극도_순함', '자극도_보통', '자극도_느껴짐'],
      dtype='object')

In [501]:
df_0824_10 = df_0824_10[['상품명', '평가', 
                         '리뷰수', '평균평점', 
                         '5점비율', '4점비율', '3점비율', '2점비율', '1점비율',
                         '피부타입_건성_좋은비율', '피부타입_복합성_좋은비율', '피부타입_지성_좋은비율', 
                         '피부고민_보습_좋은비율', '피부고민_진정_좋은비율', '피부고민_주름/미백_좋은비율', 
                         '자극도_순함', '자극도_보통', '자극도_느껴짐', '태그',
                         '리뷰어정보', '작성날짜', '평점', 
                         '피부타입', '피부고민', '자극도', 
                         '리뷰', '도움수']]

In [489]:
df_0824_10

,상품명,평가,리뷰수,평균평점,5점비율,4점비율,3점비율,2점비율,1점비율,피부타입_건성_좋은비율,...,자극도_느껴짐,태그,리뷰어정보,작성날짜,평점,피부타입,피부고민,자극도,리뷰,도움수
0,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건",4.6점,75%,18%,5%,1%,1%,20%,...,2%,"['ALL', '8809738592137|여름', '8809738592137|여드름...",화장품분석녀\nTOP 400\n메이크업 분야 탑리뷰어\n복합성\n웜톤\n각질\n모공,2021.08.10,5점만점에 5점,복합성에 좋아요,보습에 좋아요,자극없이 순해요,이브랜드는 올리브영 입점전부터 써본브랜드입니다\n수분크림구매시에 성분이 중요한데요\...,이 리뷰가 도움이 돼요! 15
1,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건",4.6점,75%,18%,5%,1%,1%,20%,...,2%,"['ALL', '8809738592137|여름', '8809738592137|여드름...",앙꼬빵\n지성\n모공\n블랙헤드,2021.08.09,5점만점에 5점,지성에 좋아요,보습에 좋아요,자극없이 순해요,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n1+1행사한다...,이 리뷰가 도움이 돼요! 8
2,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건",4.6점,75%,18%,5%,1%,1%,20%,...,2%,"['ALL', '8809738592137|여름', '8809738592137|여드름...",다블리\nTOP 50\n건강/위생용품 분야 탑리뷰어\n복합성\n웜톤\n모공\n블랙헤드,2021.07.03,5점만점에 5점,복합성에 좋아요,진정에 좋아요,자극없이 순해요,올리브영 대표 클린뷰티! 라운드랩!\n\n# 라운드랩 자작나무 수분 크림 \n\n✔...,이 리뷰가 도움이 돼요! 9
3,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건",4.6점,75%,18%,5%,1%,1%,20%,...,2%,"['ALL', '8809738592137|여름', '8809738592137|여드름...",라당\nTOP 1000\n지성\n웜톤\n각질\n모공,2021.02.18,5점만점에 5점,지성에 좋아요,보습에 좋아요,자극없이 순해요,💚수분감\n진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 1...,이 리뷰가 도움이 돼요! 11
4,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),최고,"총 1,632 건",4.6점,75%,18%,5%,1%,1%,20%,...,2%,"['ALL', '8809738592137|여름', '8809738592137|여드름...",탄산수맛있당\nTOP 900\n건성\n쿨톤\n각질\n미백,2021.08.13 매장,5점만점에 5점,복합성에 좋아요,보습에 좋아요,자극없이 순해요,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,이 리뷰가 도움이 돼요! 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14273,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),최고,총 826 건,4.8점,86%,12%,2%,0%,0%,32%,...,0%,"['ALL', '8809780880381|화농성', '8809780880381|건조...",ddoc****,2021.06.08,5점만점에 5점,복합성에 좋아요,보습에 좋아요,자극없이 순해요,이 스킨으로 스킨팩하면 확실히 피부가 촉촉해지고 맑아져요!\n※ 해당 리뷰는 원칙적...,이 리뷰가 도움이 돼요! 0
14274,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),최고,총 826 건,4.8점,86%,12%,2%,0%,0%,32%,...,0%,"['ALL', '8809780880381|화농성', '8809780880381|건조...",ch****,2021.06.08,5점만점에 5점,복합성에 좋아요,진정에 좋아요,자극없이 순해요,솔직히 받고 아직 안썼지만 맘에는 듭니다. 잘쓰겄소.\n※ 해당 리뷰는 원칙적으로 ...,이 리뷰가 도움이 돼요! 0
14275,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),최고,총 826 건,4.8점,86%,12%,2%,0%,0%,32%,...,0%,"['ALL', '8809780880381|화농성', '8809780880381|건조...",floray****,2021.06.08 매장,5점만점에 4점,건성에 좋아요,진정에 좋아요,자극없이 순해요,제가 써본 토너 중에는 가장 순하고 흡수가 잘되는것 같아요^^\n※ 해당 리뷰는 원...,이 리뷰가 도움이 돼요! 0
14276,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),최고,총 826 건,4.8점,86%,12%,2%,0%,0%,32%,...,0%,"['ALL', '8809780880381|화농성', '8809780880381|건조...",dndwl****,2021.06.07,5점만점에 5점,건성에 좋아요,보습에 좋아요,자극없이 순해요,트러블 피부인데 이거는 듬뿍 써도 아무것도 안나요!! 보습효과도 좋아서 3번째 재구...,이 리뷰가 도움이 돼요! 0


In [506]:
df_0824_10.to_csv("olive_review_10_0824.csv", index=False, encoding='utf-8-sig')

In [511]:
df_only_review = df_0824_10[['상품명', '리뷰']]

In [513]:
df_only_review.to_csv("olive_only_review_10.csv", index=False, encoding='utf-8-sig')

In [516]:
sample_50 = df_only_review.sample(n=50)

In [517]:
sample_50.to_csv("review_sample_50.csv", encoding='utf-8-sig')

In [493]:
def data_cleansing(df):
    df['리뷰수'] = df['리뷰수'][2:-2]
    if (len(df['도움수']) < 5):
        df['도움수'] = df['도움수']
    else:
        df['도움수'] = df['도움수'].split('!')[1]
        
    
    return df

In [496]:
df_0824_10['도움수'][0]

' 15'

In [475]:
for i in range(len(df_0824_10)):
    print(i)
    print(df_0824_10.iloc[i]['도움수'].split('!')[1])

0
 15
1
 8
2
 9
3
 11
4
 1
5
 7
6
 3
7
 3
8
 2
9
 2
10
 1
11
 1
12
 5
13
 2
14
 1
15
 2
16
 1
17
 0
18
 0
19
 0
20
 0
21
 0
22
 1
23
 2
24
 1
25
 2
26
 4
27
 3
28
 2
29
 9
30
 1
31
 0
32
 0
33
 1
34
 1
35
 1
36
 2
37
 0
38
 0
39
 0
40
 1
41
 0
42
 0
43
 4
44
 4
45
 0
46
 2
47
 3
48
 0
49
 0
50
 0
51
 0
52
 0
53
 2
54
 1
55
 0
56
 2
57
 0
58
 1
59
 0
60
 0
61
 0
62
 0
63
 0
64
 0
65
 0
66
 0
67
 1
68
 0
69
 0
70
 0
71
 0
72
 0
73
 1
74
 0
75
 0
76
 0
77
 0
78
 0
79
 0
80
 0
81
 2
82
 0
83
 0
84
 1
85
 4
86
 4
87
 1
88
 0
89
 0
90
 0
91
 0
92
 0
93
 0
94
 0
95
 0
96
 1
97
 0
98
 2
99
 0
100
 0
101
 0
102
 1
103
 0
104
 0
105
 0
106
 0
107
 0
108
 0
109
 3
110
 0
111
 0
112
 0
113
 0
114
 0
115
 0
116
 0
117
 0
118
 0
119
 0
120
 0
121
 0
122
 6
123
 0
124
 0
125
 0
126
 0
127
 0
128
 0
129
 0
130
 0
131
 3
132
 0
133
 0
134
 0
135
 0
136
 0
137
 0
138
 0
139
 0
140
 2
141
 0
142
 0
143
 0
144
 0
145
 0
146
 0
147
 0
148
 0
149
 0
150
 0
151
 0
152
 0
153
 0
154
 0
155
 0
156
 0
157
 0
15

 0
1429
 0
1430
 0
1431
 0
1432
 0
1433
 0
1434
 0
1435
 0
1436
 0
1437
 0
1438
 0
1439
 0
1440
 0
1441
 1
1442
 2
1443
 1
1444
 1
1445
 1
1446
 1
1447
 0
1448
 0
1449
 0
1450
 0
1451
 0
1452
 0
1453
 0
1454
 0
1455
 0
1456
 0
1457
 0
1458
 0
1459
 0
1460
 0
1461
 0
1462
 0
1463
 0
1464
 0
1465
 0
1466
 0
1467
 0
1468
 0
1469
 0
1470
 0
1471
 0
1472
 0
1473
 0
1474
 0
1475
 0
1476
 0
1477
 0
1478
 0
1479
 0
1480
 0
1481
 0
1482
 0
1483
 0
1484
 0
1485
 0
1486
 0
1487
 0
1488
 0
1489
 0
1490
 0
1491
 0
1492
 0
1493
 0
1494
 0
1495
 0
1496
 0
1497
 0
1498
 0
1499
 0
1500
 0
1501
 0
1502
 0
1503
 0
1504
 0
1505
 0
1506
 0
1507
 0
1508
 0
1509
 0
1510
 0
1511
 0
1512
 0
1513
 0
1514
 0
1515
 0
1516
 0
1517
 0
1518
 0
1519
 0
1520
 0
1521
 0
1522
 0
1523
 0
1524
 0
1525
 0
1526
 0
1527
 0
1528
 0
1529
 0
1530
 0
1531
 0
1532
 0
1533
 0
1534
 0
1535
 0
1536
 0
1537
 0
1538
 0
1539
 0
1540
 0
1541
 0
1542
 0
1543
 0
1544
 0
1545
 0
1546
 0
1547
 0
1548
 0
1549
 0
1550
 0
1551
 0
1552
 0
1553


 0
2504
 0
2505
 0
2506
 0
2507
 0
2508
 0
2509
 0
2510
 0
2511
 0
2512
 0
2513
 0
2514
 0
2515
 0
2516
 0
2517
 0
2518
 0
2519
 0
2520
 0
2521
 0
2522
 0
2523
 0
2524
 0
2525
 0
2526
 0
2527
 0
2528
 0
2529
 0
2530
 0
2531
 0
2532
 0
2533
 0
2534
 0
2535
 0
2536
 0
2537
 1
2538
 0
2539
 0
2540
 0
2541
 0
2542
 0
2543
 0
2544
 0
2545
 0
2546
 0
2547
 0
2548
 0
2549
 0
2550
 0
2551
 0
2552
 0
2553
 0
2554
 0
2555
 0
2556
 0
2557
 0
2558
 0
2559
 0
2560
 0
2561
 0
2562
 0
2563
 0
2564
 0
2565
 0
2566
 0
2567
 0
2568
 0
2569
 0
2570
 0
2571
 0
2572
 0
2573
 0
2574
 1
2575
 1
2576
 1
2577
 1
2578
 1
2579
 1
2580
 0
2581
 0
2582
 0
2583
 0
2584
 0
2585
 0
2586
 0
2587
 0
2588
 1
2589
 1
2590
 1
2591
 1
2592
 0
2593
 0
2594
 0
2595
 0
2596
 0
2597
 0
2598
 0
2599
 0
2600
 0
2601
 0
2602
 0
2603
 0
2604
 0
2605
 0
2606
 0
2607
 0
2608
 0
2609
 1
2610
 0
2611
 0
2612
 1
2613
 1
2614
 0
2615
 0
2616
 0
2617
 0
2618
 0
2619
 0
2620
 0
2621
 0
2622
 0
2623
 0
2624
 0
2625
 0
2626
 0
2627
 0
2628


 0
4044
 0
4045
 0
4046
 0
4047
 0
4048
 0
4049
 0
4050
 0
4051
 0
4052
 0
4053
 0
4054
 0
4055
 0
4056
 0
4057
 0
4058
 0
4059
 0
4060
 0
4061
 0
4062
 0
4063
 0
4064
 0
4065
 0
4066
 0
4067
 0
4068
 0
4069
 0
4070
 0
4071
 0
4072
 0
4073
 0
4074
 0
4075
 0
4076
 0
4077
 0
4078
 0
4079
 0
4080
 0
4081
 0
4082
 0
4083
 0
4084
 0
4085
 0
4086
 0
4087
 0
4088
 0
4089
 0
4090
 0
4091
 0
4092
 0
4093
 0
4094
 0
4095
 0
4096
 0
4097
 0
4098
 0
4099
 1
4100
 0
4101
 2
4102
 1
4103
 2
4104
 1
4105
 1
4106
 1
4107
 1
4108
 0
4109
 0
4110
 0
4111
 0
4112
 0
4113
 1
4114
 1
4115
 1
4116
 1
4117
 1
4118
 1
4119
 1
4120
 1
4121
 1
4122
 1
4123
 1
4124
 1
4125
 1
4126
 1
4127
 1
4128
 1
4129
 1
4130
 1
4131
 1
4132
 1
4133
 1
4134
 1
4135
 1
4136
 1
4137
 1
4138
 1
4139
 1
4140
 1
4141
 1
4142
 1
4143
 1
4144
 1
4145
 1
4146
 1
4147
 1
4148
 1
4149
 1
4150
 1
4151
 1
4152
 1
4153
 1
4154
 1
4155
 1
4156
 0
4157
 1
4158
 0
4159
 0
4160
 0
4161
 0
4162
 0
4163
 0
4164
 0
4165
 0
4166
 0
4167
 0
4168


5469
 1
5470
 1
5471
 1
5472
 1
5473
 1
5474
 1
5475
 1
5476
 1
5477
 1
5478
 1
5479
 1
5480
 1
5481
 1
5482
 1
5483
 1
5484
 1
5485
 1
5486
 1
5487
 1
5488
 1
5489
 1
5490
 1
5491
 1
5492
 1
5493
 1
5494
 1
5495
 1
5496
 1
5497
 1
5498
 1
5499
 1
5500
 1
5501
 1
5502
 1
5503
 1
5504
 1
5505
 1
5506
 1
5507
 1
5508
 1
5509
 1
5510
 1
5511
 1
5512
 1
5513
 1
5514
 1
5515
 1
5516
 1
5517
 1
5518
 1
5519
 1
5520
 1
5521
 1
5522
 1
5523
 1
5524
 1
5525
 1
5526
 0
5527
 0
5528
 0
5529
 0
5530
 0
5531
 0
5532
 0
5533
 0
5534
 0
5535
 0
5536
 0
5537
 0
5538
 0
5539
 0
5540
 0
5541
 0
5542
 0
5543
 0
5544
 1
5545
 1
5546
 1
5547
 1
5548
 1
5549
 1
5550
 1
5551
 1
5552
 1
5553
 1
5554
 1
5555
 1
5556
 1
5557
 1
5558
 1
5559
 1
5560
 1
5561
 1
5562
 1
5563
 1
5564
 1
5565
 1
5566
 1
5567
 1
5568
 1
5569
 1
5570
 1
5571
 1
5572
 1
5573
 0
5574
 0
5575
 0
5576
 1
5577
 1
5578
 1
5579
 1
5580
 1
5581
 1
5582
 1
5583
 1
5584
 1
5585
 1
5586
 1
5587
 1
5588
 1
5589
 1
5590
 1
5591
 1
5592
 1
5593
 1


IndexError: list index out of range

In [476]:
df_0824_10.iloc[6090]['도움수']

'자극도'

In [494]:
data_10 = df_0824_10.apply(data_cleansing, axis=1)

TypeError: ("object of type 'float' has no len()", 'occurred at index 7352')

In [507]:
data_10

NameError: name 'data_10' is not defined

In [ ]:
def reviewer_split(df):
    df['닉네임'] = df['리뷰어정보'].split('\n')[0]
    if (df['리뷰어정보'].split('\n')[1][0] ==  'T'):
        df['리뷰어랭킹'] = df['리뷰어정보'].split('\n')[1][0]
    else :
        df['리뷰어_피부타입'] = df['리뷰어정보'].split('\n')[1][0]
        
    
    sample = df['리뷰정보'].split('\n')[8:-2]
    df['리뷰'] = '\n'.join(sample)
    df['도움수'] = df['리뷰정보'].split('\n')[-2]
    return df